---
title : "Fragility Curve Overtopping"
execute:
    output: asis
---

Voor het berekenen van een fragility curve voor het mechanisme GEKB wordt gebruikt gemaakt van de [`pydra_core`](https://github.com/HKV-products-services/pydra_core) module, meer documentatie over de `pydra_core` module is [hier](https://hkv-products-services.github.io/pydra_core/) te vinden. 

De module wordt gebruikt om een overslag debiet te berekenen voor verschillende omstandigheden en op basis hiervan wordt een fragility curve opgesteld.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

from toolbox_continu_inzicht.fragility_curves import (
    FragilityCurveOvertoppingWaveData,
)

In [ ]:
from toolbox_continu_inzicht.base.data_adapter import Config, DataAdapter

path = Path.cwd() / "data_sets" / "8.fragility_curve_overtopping_wavedata"
config = Config(config_path=path / "test_fragility_curve_overtopping_wavedata.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

De volgende configuratie wordt gebruikt voor het maken van een GEKB curve

```yaml
GlobalVariables:
    rootdir: "data_sets/8.fragility_curve_overtopping_wavedata"
    moments: [ -24, 0, 24, 48 ]
    logging:
        level: INFO
    FragilityCurveOvertoppingWaveData:
        closing_situation: 1

DataAdapter:
    default_options:
        csv:
            sep: ","
    slopes:
        type: csv
        file: "slopes_5.csv"
    profiles:
        type: csv
        file: "profiles_new.csv"
    waveval_uncert:
        type: csv
        file: "waveval_uncert_5.csv"
        index_col: false
    waveval_id:
        type: csv
        file: "waveval_id_5.csv"
        index_col: false
    waveval:
        type: csv
        file: "waveval_5.csv"
        index_col: false
    fragility_curves:
        type: csv
        file: "fragility_curves_5.csv"


```

Er zijn vijf verschillende data bestanden nodig.<br>
Ten eerste een tabel met de informatie over de helling (slope) waarvoor de curve wordt afgeleid.


In [ ]:
data_adapter.input("slopes")

Ten tweede informatie over de hoogte van de kruin, de orientatie, of er een dam is, maatgevende windsnelheid enz.


In [ ]:
data_adapter.input("profiles")

Ten derde de onzekerheden van de golfparameters per HR-locatie (modelonzekerheden).


In [ ]:
data_adapter.input("waveval_uncert")

Ten vierde de index voor de golfcombinaties (per windrichting en windsnelheid) met het type golfparameter.


In [ ]:
data_adapter.input("waveval_id")

Ten vijfde de daadwerkelijke golfwaarden per waterstand, gekoppeld aan de golfcombinaties via `waveval_id`.


In [ ]:
data_adapter.input("waveval").head(4)  # (ingekorte versie)

Deze informatie wordt mee gegeven aan de FragilityCurveOvertoppingWaveData voor het genereren van een curve.


In [ ]:
fragility_curve_overtopping = FragilityCurveOvertoppingWaveData(
    data_adapter=data_adapter
)
fragility_curve_overtopping.run(
    input=[
        "slopes",
        "profiles",
        "waveval_uncert",
        "waveval_id",
        "waveval",
    ],
    output="fragility_curves",
)

Deze curve kunnen we vervolgens weergeven:


In [ ]:
fig, ax = plt.subplots()
fragility_curve_overtopping.as_dataframe().set_index("hydraulicload").plot(ax=ax)
ax.set_ylabel("Failure probability");

In [ ]:
data_adapter.input("fragility_curves")